    In this exercise, we are to going to create our own public key and private key for Elgamal cryptosystem. 
    But to create these key we need to find a safe prime number more than 400 digits.
    So first we find this number then we can create our public key and secret key.

    Were are going to use the Miller Rabin test to test if a candidate q is probably prime then test if p=2*q + 1 is always probably prime.  

In [104]:
import random, math, sympy
from math import isqrt

## Get the number of bits necessary 

In [105]:
def bits_number(d: int) -> int:
    '''
        return the number of bits for a number with d digits
    '''
    log2_10 = math.log2(10)
    return math.floor(d * log2_10)

In [106]:
bits_number(400)

1328

## Miller Rabin Test

In [2]:
def miller_rabin(n: int, k=5) -> bool:
    ''' Return True if the number is probabily prime and False if it is a composite '''
    
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
        
    # Write n-1 as 2^s*d
    s, d = 0, n-1
    while d%2 == 0:
        d //= 2
        s += 1

    def is_composite(a):
        if pow(a, d, n) == 1:
            return False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                return False
        return True

    # Test 
    for _ in range(k):
        a = random.randrange(2, n-1)
        if is_composite(a):
            return False

    return True


def isPrime(n: int) -> bool:
    '''
        return True if a number n is prime.
    '''
    small_primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]
    if n in small_primes:
        return True
    if any(n % p == 0 for p in small_primes):
        return False
    return miller_rabin(n)

## Safe Prime Number

In [108]:
def sieve_of_eratosthenes(limit: int) -> list:
    """
    Generate all prime numbers up to a given limit using the Sieve of Eratosthenes.
    """
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False  # 0 and 1 are not prime
    for start in range(2, isqrt(limit) + 1):
        if sieve[start]:
            for multiple in range(start * start, limit + 1, start):
                sieve[multiple] = False
    return [num for num, is_prime in enumerate(sieve) if is_prime]

def is_divisible(candidate: int, small_primes: list):
    """
    Check if a candidate is divisible by any of the small primes.
    """
    for p in small_primes:
        if candidate % p == 0:
            return True
    return False

def generate_safe_prime_number(n: int, small_prime_limit=10000):
    """
    Generate a safe prime number of at least n bits such that p = 2q + 1 and q are both prime.
    """
    # Generate all small primes up to the limit
    small_primes = sieve_of_eratosthenes(small_prime_limit)

    while True:
        # Generate a random candidate q with n-1 bits
        q = random.randrange(2**(n - 1), 2**n)
        
        # Eliminate candidates divisible by small primes
        if is_divisible(q, small_primes):
            continue

        # Test if q is probably prime
        if not isPrime(q):
            continue

        # Compute p = 2q + 1
        p = 2 * q + 1

        # Test if p is probably prime
        if isPrime(p):
            return p

In [109]:
p = generate_safe_prime_number(1328)
p

11137477182687017275715874435161269660442861563519013799203632949546781285414260206149130941197410948559236918524852801438538753593774982445364895410650162670604769349973173963793257813347707746352688685195617505092080317363084174046435011907611423949612667364398981624079643111475547171407643621430748350383508788057962243573553384664768425539880680759065974217603422483841449181599264333954865761479

In [110]:
q = (p-1)//2
q

5568738591343508637857937217580634830221430781759506899601816474773390642707130103074565470598705474279618459262426400719269376796887491222682447705325081335302384674986586981896628906673853873176344342597808752546040158681542087023217505953805711974806333682199490812039821555737773585703821810715374175191754394028981121786776692332384212769940340379532987108801711241920724590799632166977432880739

In [111]:
isPrime(q)

True

## Public and Secret Keys
    Now, we have a safe prime number p more than 400 digits. So we work with the cyclic group G = (Z/pZ)*.
    The public keys is g^a such that g is a generator of this group K and a is an element of the group.
    Let's find a generator for (Z/pZ)*

### Steps to Find a Generator of (Z/pZ)*:

1. **Properties of Safe Primes**:
   - If p = 2q + 1, where q is prime, then (Z/pZ)* is a cyclic group of order p-1 = 2q.
   - The generators are elements g in (Z/pZ)* such that the successive powers of g generate the entire group.

2. **Conditions for g to be a Generator**:
   - g is a generator of (Z/pZ)* if and only if g^((p-1)/2) is different with 1(modp) for all divisors d of p-1, except p-1.

3. **For a Safe Prime p**:
   - The divisors of p-1 = 2q are 1, 2, q, and 2q. Thus:
     - Verify that g^((p-1)/2) is different with 1(modp) \not\equiv 1 \mod p\) (to eliminate elements of order 2).
     - Verify that g^((p-1)/q) is different with 1(modp)(to eliminate elements of order q).

In [124]:
def find_generator(p: int) -> int:
    """
    Find a generator of the multiplicative group (Z/pZ)*, where p is a safe prime.
    """
    if p <= 2 or (p - 1) % 2 != 0:
        raise ValueError("p must be a safe prime, such that p = 2q + 1 where q is prime.")
    
    # Compute q such that p = 2q + 1
    q = (p - 1) // 2

    # Check if q is prime
    if not isPrime(q):
        raise ValueError("p is not a safe prime.")

    # Test random candidates for being a generator
    while True:
        # Random candidate g in the range [2, p-1]
        g = random.randint(2, p - 2)

        # Check the generator conditions
        if pow(g, (p - 1) // 2, p) != 1 and pow(g, (p - 1) // q, p) != 1:
            return g


In [125]:
g = find_generator(p)
g

10724962864037742874898186261502114135831287039308933918388402142152292879200680300447536276143528195922420382400455330693856200030895309523187168420489949155667201085919446084125197504776832477044728858111493837562988266763294649159899545227220673566068889568589477119624583488322646947064852037382589582367809595535392010769666038211640867355018361958387945852160890496749721060294247264255959323992

In [143]:
def public_and_secret_key(p: int, g: int) -> dict:
    ''' p: a prime number for modulos calculus
        g: generator of G = (Z/pZ)*
        return a dict with key: public_key, secret_key
    '''
    #order of the cyclic group
    n = p-1
    
    #Generate a secret key
    secret_key = random.randrange(2, n)

    #Get the public key
    public_key = (p, g, pow(g, secret_key, n))

    result = {
        'public_key': public_key,
        'secret_key': secret_key
    }
    return result

In [144]:
public_key = public_and_secret_key(p, g).get('public_key')
secret_key = public_and_secret_key(p, g).get('secret_key')

In [145]:
public_key

(11137477182687017275715874435161269660442861563519013799203632949546781285414260206149130941197410948559236918524852801438538753593774982445364895410650162670604769349973173963793257813347707746352688685195617505092080317363084174046435011907611423949612667364398981624079643111475547171407643621430748350383508788057962243573553384664768425539880680759065974217603422483841449181599264333954865761479,
 10724962864037742874898186261502114135831287039308933918388402142152292879200680300447536276143528195922420382400455330693856200030895309523187168420489949155667201085919446084125197504776832477044728858111493837562988266763294649159899545227220673566068889568589477119624583488322646947064852037382589582367809595535392010769666038211640867355018361958387945852160890496749721060294247264255959323992,
 15990356418314100105446565535143848004233726697778352488933276851037528258053805098100910479013376944395269773314368334037241386154285800010448952645737611784243224015959349609207001170633779

In [146]:
secret_key

6541890871780170598530330343010516454890771624069829222342854498526931504518395967723181493896804397200161908589883259976280597293507145347998155764722695302784424137936656038920970366941369370880945555921609669112253569540505590225153171432570745205281796396233861683365788587381166166095788659069042462384967660614568111229356068578738461883927008073872632606659785649327297947276628087697876385793